In [1]:
# !pip install pyautogen
# !pip install openai

# or pip install -r requirements.txt (from the github repo)

In [2]:
# source: https://github.com/microsoft/autogen/blob/main/notebook/autobuild_agent_library.ipynb
config_file_or_env = './OAI_CONFIG_LIST'
default_llm_config = {
    'temperature': 0
}

In [3]:
# Create AgentBuilder instance using config path & default config
# Specify  builder & agent models
from autogen.agentchat.contrib.agent_builder import AgentBuilder
builder = AgentBuilder(
    config_file_or_env=config_file_or_env, 
    builder_model='gpt-4')

In [4]:
# Specify the building tasks
# define building taskwith description *& examples
# helps bild manager decide on agents
# example task: "Create a multi-agent system for task X"

building_task = """Create a multi-agent system for the task of finding papers on arxiv about artificial intelligence, and
analyse its application in some domain. For example, find a latest paper about gpt-4 on arxiv and find its potential applications in software."""

In [5]:
# use build() method to generate agents
# include a user proxy for tasks involving coding

agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

==> Generating agents...
['arxiv_AI_paper_search_engineer', 'AI_application_analyst', 'gpt4_technology_specialist', 'software_domain_expert', 'report_writer'] are generated.
==> Generating system message...
Preparing system message for arxiv_AI_paper_search_engineer
Preparing system message for AI_application_analyst
Preparing system message for gpt4_technology_specialist
Preparing system message for software_domain_expert
Preparing system message for report_writer
==> Generating description...
Preparing description for arxiv_AI_paper_search_engineer
Preparing description for AI_application_analyst
Preparing description for gpt4_technology_specialist
Preparing description for software_domain_expert
Preparing description for report_writer
==> Creating agents...
Creating agent arxiv_AI_paper_search_engineer with backbone gpt-4...
Creating agent AI_application_analyst with backbone gpt-4...
Creating agent gpt4_technology_specialist with backbone gpt-4...
Creating agent software_domain_exp

In [6]:
agent_list

In [7]:
agent_configs

{'building_task': 'Create a multi-agent system for the task of finding papers on arxiv about artificial intelligence, and\nanalyse its application in some domain. For example, find a latest paper about gpt-4 on arxiv and find its potential applications in software.',
 'agent_configs': [{'name': 'arxiv_AI_paper_search_engineer',
   'model': 'gpt-4',
   'system_message': 'You are now in a group chat. You need to complete a task with other participants. As an "arxiv_AI_paper_search_engineer", your main task involves creating a multi-agent system for finding artificial intelligence related papers on arxiv and analyzing their application in specified domains. You have to demonstrate both your technical understanding and practical knowledge about AI in various applications, such as the potential use of GPT-4 in software. \n\nYour tasks may include identifying latest papers, using Python programs for data extraction and information retrieval, or discussing the potential implications of AI res

In [8]:
# Execute the task
# agents collaborate in a group chat to complete task
# combine llms, human inputs, and tools
# example: start_task(execution_task, agent_list, llm_config)
from autogen import config_list_from_json, GroupChat, GroupChatManager

config_list = config_list_from_json(config_file_or_env)

def start_task(execution_task: str, agent_list: list):
    group_chat = GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list, **default_llm_config})
    agent_list[0].initiate_chat(manager, message=execution_task)


start_task(
    execution_task="Find a recent paper about gpt-4 on arxiv.",
    agent_list=agent_list,
)

User_console_and_code_interpreter (to chat_manager):

Find a recent paper about gpt-4 on arxiv.

--------------------------------------------------------------------------------
arxiv_AI_paper_search_engineer (to chat_manager):

Sure, I will write a Python script to scrape the arxiv website for recent papers related to GPT-4. The script will use the arxiv API to search for papers and BeautifulSoup to parse the HTML. 

Here is the Python script:

```python
# filename: arxiv_scraper.py

import requests
from bs4 import BeautifulSoup
import time

def search_arxiv(query, max_results=10):
    base_url = "http://export.arxiv.org/api/query?"
    query = "all:" + query
    start = 0
    total_results = 1  # this is just to initialize the while loop
    papers = []

    while len(papers) < total_results and len(papers) < max_results:
        response = requests.get(base_url,
                                params={
                                    "search_query": query,
                      

In [9]:
builder.clear_all_agents()

All agents have been cleared.


In [10]:
saved_path = builder.save()

Building config saved to ./save_config_77e9e37d867631970f8eebebaf301168.json


Configurations will be saved in JSON format like such:

```
// FILENAME: save_config_TASK_MD5.json
{
"building_task": "Find a paper on arxiv about artificial intelligence, 
and analyze its application in some domain. For example, find a latest 
paper about gpt-4 on arxiv and find its potential applications in 
software." ,
"agent_configs": [
{
    "name": "...",
    .....
},
...
],
"manager_system_message": "...",
"code_execution_config": {...},
"default_llm_config": {...}
}

```